第63題
當對手失誤掛網時，分析前一拍周天成出球球種與落點特徵。

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定，確保中文字正常顯示
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據存在
if len(df) > 0:
    # 1. 篩選出對手失誤掛網的記錄
    opponent_net_faults = df[(df['lose_reason'] == '掛網') & (df['getpoint_player'] == 'CHOU Tien Chen')]

    # 2. 使用 shift 獲取前一拍周天成的出球紀錄
    opponent_net_faults = opponent_net_faults.copy()
    opponent_net_faults['prev_player'] = opponent_net_faults.groupby(['match_id', 'set', 'rally'])['player'].shift(1)
    opponent_net_faults['prev_type'] = opponent_net_faults.groupby(['match_id', 'set', 'rally'])['type'].shift(1)
    opponent_net_faults['prev_landing_area'] = opponent_net_faults.groupby(['match_id', 'set', 'rally'])['landing_area'].shift(1)
    
    # 3. 篩選出周天成的出球球種和落點
    chou_tien_chen_shots = opponent_net_faults[opponent_net_faults['prev_player'] == 'CHOU Tien Chen']

    # 檢查是否有有效的周天成出球紀錄
    if not chou_tien_chen_shots.empty:
        # 4. 統計出球球種與落點的分布
        shot_type_distribution = chou_tien_chen_shots['prev_type'].value_counts(normalize=True)
        landing_area_distribution = chou_tien_chen_shots['prev_landing_area'].value_counts(normalize=True)

        # 打印統計資訊
        print("周天成的出球球種分布（對手失誤掛網前）：")
        print(shot_type_distribution)
        print("\n周天成的落點區域分布（對手失誤掛網前）：")
        print(landing_area_distribution)
        
        # 5. 繪製可視化圖表
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))
        
        # 出球球種分布
        ax[0].bar(shot_type_distribution.index, shot_type_distribution.values)
        ax[0].set_title("周天成出球球種分布")
        ax[0].set_xlabel('球種')
        ax[0].set_ylabel('比例')
        ax[0].set_xticks(range(len(shot_type_distribution.index)))
        ax[0].set_xticklabels(shot_type_distribution.index, rotation=45, ha="right")
        
        # 落點區域分布
        ax[1].bar(landing_area_distribution.index, landing_area_distribution.values, color='orange')
        ax[1].set_title("周天成落點區域分布")
        ax[1].set_xlabel('落點區域')
        ax[1].set_ylabel('比例')
        ax[1].set_xticks(range(len(landing_area_distribution.index)))
        ax[1].set_xticklabels(landing_area_distribution.index, rotation=45, ha="right")
        
        plt.tight_layout()
    else:
        print("沒有找到周天成的有效出球紀錄。")

else:
    print("資料數據缺失，無法進行分析。")

第64題
分析周天成與對手在高壓球（平推、平抽）交換中誰先失誤的統計。

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

# 設定字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 假設df已經定義
# 1. 過濾高壓球的數據
high_pressure_types = ['平球', '推撲球']
high_pressure_df = df[df['type'].isin(high_pressure_types)]

# 2. 確認有沒有數據
if len(high_pressure_df) > 0:
    # 3. 統計雙方失誤次數
    rounds_with_mistakes = []

    for (match_id, set_id, rally_id), group in high_pressure_df.groupby(['match_id', 'set', 'rally']):
        if group['getpoint_player'].notnull().any():
            last_shot_player = group.iloc[-1]['player']  # 最後一球的player
            point_player = group.iloc[-1]['getpoint_player']  # 得分者

            if last_shot_player != point_player:  # 確認是不是自己失誤
                # 若 getpoint_player 與球員不同，則球員失誤
                losing_player = last_shot_player
                rounds_with_mistakes.append(losing_player)

    # 計算失誤次數
    mistake_count = Counter(rounds_with_mistakes)
    print("失誤計數：", mistake_count)

    # 4. 計算失誤率
    total_rounds = high_pressure_df.groupby(['match_id', 'set', 'rally']).ngroups
    player_mistake_rate = mistake_count['Kento MOMOTA'] / total_rounds
    opponent_mistake_rate = mistake_count['CHOU Tien Chen'] / total_rounds

    # 打印失誤率
    print("Kento MOMOTA 的失誤率：", player_mistake_rate)
    print("CHOU Tien Chen 的失誤率：", opponent_mistake_rate)

    # 圖表展示失誤次數
    labels = ['Kento MOMOTA', 'CHOU Tien Chen']
    mistakes = [mistake_count['Kento MOMOTA'], mistake_count['CHOU Tien Chen']]

    fig, ax = plt.subplots()
    ax.bar(labels, mistakes, color=['blue', 'orange'])
    ax.set_title('高壓球交換中的失誤次數')
    ax.set_ylabel('失誤次數')
    plt.tight_layout()
else:
    print("沒有高壓球類型的數據。")

第65題
統計周天成所有「得分帶有連續性」的回合（連 2 分以上）的常見戰術組合。

In [ ]:
import platform
import pandas as pd
import matplotlib.pyplot as plt

# 設定字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確認數據集不為空
if len(df) == 0:
    raise ValueError("DataFrame is empty.")

# 填充必要的遺失值
df['getpoint_player'] = df['getpoint_player'].fillna(method='ffill')

# step1: 識別連續得分
# 找出周天成得分的回合
df_ctc_points = df[df['getpoint_player'] == 'CHOU Tien Chen']
df_ctc_points['prev_rally_winner'] = df_ctc_points.groupby(['match_id', 'set'])['getpoint_player'].shift(1)

# 請注意：連續得分定義為連續兩次或多次周天成得分
df_ctc_points['is_continuous'] = (df_ctc_points['prev_rally_winner'] == 'CHOU Tien Chen')
continuous_scores = df_ctc_points[df_ctc_points['is_continuous']]

# 確認找到的回合序列
print("連續得分的回合序列:")
print(continuous_scores['rally_id'].unique())

# step2: 分析戰術特徵
# 統計擊球類型
shot_types = continuous_scores['type'].value_counts()
print("擊球類型統計:")
print(shot_types)

# step3: 對手反應
# 對手在連續得分回合中的擊球類型（前一拍）
df['prev_player_type'] = df.groupby(['match_id', 'set', 'rally'])['type'].shift(1)
continuous_scores = continuous_scores.merge(df[['rally_id', 'prev_player_type']], on='rally_id', how='left')

opponent_reactions = continuous_scores['prev_player_type'].value_counts()
print("對手反應（上次擊球類型）:")
print(opponent_reactions)

# step4: 視覺化結果
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# 繪製擊球類型統計
ax[0].bar(shot_types.index, shot_types.values)
ax[0].set_title('周天成連續得分中常見的擊球類型')
ax[0].set_ylabel('次數')
ax[0].set_xlabel('擊球類型')
ax[0].set_xticklabels(shot_types.index, rotation=45)

# 繪製對手反應
ax[1].bar(opponent_reactions.index, opponent_reactions.values, color='orange')
ax[1].set_title('對手反應類型（上次擊球類型）')
ax[1].set_ylabel('次數')
ax[1].set_xlabel('對手上一次擊球類型')
ax[1].set_xticklabels(opponent_reactions.index, rotation=45)

plt.tight_layout()
plt.show()

第66題
當周天成被迫後仰擊球時，他的失誤率 vs 得分率比較。

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 驗證資料長度
if len(df) == 0:
    raise ValueError("資料集為空！")

# 選出周天成的擊球數據並使用高位擊球作為後仰擊球的指標
chou_df = df[(df['player'] == 'CHOU Tien Chen') & (df['hit_height'] == 2)]

# 計算總共符合條件的擊球次數
total_shots = len(chou_df)

# 計算得分次數: player == getpoint_player
winning_shots = len(chou_df[chou_df['player'] == chou_df['getpoint_player']])

# 使用失誤原因來計算失誤次數
losing_shots = len(chou_df[~chou_df['lose_reason'].isna()])

# 計算得分率與失誤率
winning_rate = winning_shots / total_shots if total_shots else 0
losing_rate = losing_shots / total_shots if total_shots else 0

# Print the results
print("總擊球次數:", total_shots)
print("得分次數:", winning_shots)
print("失誤次數:", losing_shots)
print("得分率:", winning_rate)
print("失誤率:", losing_rate)

# 繪製長條圖來比較得分率與失誤率
labels = ['得分率', '失誤率']
rates = [winning_rate, losing_rate]

fig, ax = plt.subplots()
ax.bar(labels, rates, color=['green', 'red'])
ax.set_ylabel('比率')
ax.set_title('周天成後仰擊球的得分率與失誤率')

# 確保視覺化不重疊
plt.tight_layout()

第67題
在被動情況下（跨步救球），下一拍攻擊性球種的成功機率。

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 設置字體，確保繁體字顯示正確
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 假設 df 已經被定義且有效
# df = pd.read_csv('your_data_file.csv') # 若有需要載入資料可放入，但此處假設 df 已存在

# 檢查數據量
if len(df) == 0:
    print("No data available.")
else:
    # 為了應用 shift，我們首先確保數據完整
    df = df.dropna(subset=['player_type', 'getpoint_player'])

    # 建立位移的 player_type 和 player 欄位
    df['next_player_type'] = df.groupby(['match_id', 'set', 'rally'])['player_type'].shift(-1)
    df['next_player'] = df.groupby(['match_id', 'set', 'rally'])['player'].shift(-1)

    # 過濾被動防守球
    passive_defense = df[(df['player_type'] == 10) & (df['next_player'] != df['player'])]

    # 選擇攻擊性球種 (接著被動球的下一拍)
    aggressive_attacks = passive_defense[(passive_defense['next_player_type'].isin([4, 9]))]  # 4:殺球, 9:推撲球

    # 找出成功的攻擊性球種得分
    successful_attacks = aggressive_attacks[aggressive_attacks['getpoint_player'] == aggressive_attacks['player']]

    # 計算成功率
    success_rate = len(successful_attacks) / len(aggressive_attacks) if len(aggressive_attacks) > 0 else 0
    print(f"被動救球後攻擊性球種的成功機率是: {success_rate:.2%}")

    # 打印細節
    print(f"總共有 {len(passive_defense)} 次被動防守。")
    print(f"其中有 {len(aggressive_attacks)} 次緊接著打出了攻擊性球種。")
    print(f"成功得分的有 {len(successful_attacks)} 次。")

第68題
分析周天成對不同對手攻擊方式差異。

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 設置字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據不為空
assert len(df) > 0, "DataFrame is empty."

# 篩選出 CHOU Tien Chen 作為擊球球員的數據
chou_df = df[df['player'] == 'CHOU Tien Chen']

# 刪除掉擊球結果為 NaN 的行以避免對結果數據影響
chou_df = chou_df.dropna(subset=['getpoint_player'])

# 使用 `type` 欄位來分析不同對手情況下周天成的攻擊類型和頻率
attack_types = chou_df.groupby(['opponent', 'type']).size().unstack()

# 填充缺失值 0，以便於後續的計算和繪圖
attack_types = attack_types.fillna(0)

# 計算每種攻擊類型的得分次數
win_by_type = chou_df[chou_df['player'] == chou_df['getpoint_player']].groupby(['opponent', 'type']).size().unstack()

# 填充缺失值 0
win_by_type = win_by_type.fillna(0)

# 計算每種攻擊類型的成功率
success_rate_by_type = (win_by_type / attack_types).fillna(0)

# 可視化展示
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 10))

# 攻擊類型頻率
attack_types.plot(kind='bar', ax=axes[0])
axes[0].set_title('周天成的攻擊類型頻率對不同對手')
axes[0].set_ylabel('次數')
axes[0].legend(title='攻擊類型')

# 攻擊類型的成功率
success_rate_by_type.plot(kind='bar', ax=axes[1])
axes[1].set_title('周天成的攻擊類型成功率對不同對手')
axes[1].set_ylabel('成功率')
axes[1].legend(title='攻擊類型')

plt.tight_layout()

# 顯示結果
fig

第69題
計算周天成「放短球後被撲殺」的比例，以及是否在比賽後段上升。

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 設置字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據存在且不為空
if len(df) > 0:
    # 篩選 CHOU Tien Chen 網前球（放短球）的資料
    chou_short_netplay_df = df[(df['player'] == 'CHOU Tien Chen') & (df['type'] == '網前球')]

    # 為了確認後一球對手撲殺，進行 shift(-1) 操作以取得對手行為
    chou_short_netplay_df['opponent_next_type'] = chou_short_netplay_df.groupby(['match_id', 'set', 'rally'])['type'].shift(-1)
    chou_short_netplay_df['opponent_next_player'] = chou_short_netplay_df.groupby(['match_id', 'set', 'rally'])['player'].shift(-1)
    chou_short_netplay_df['opponent_next_getpoint_player'] = chou_short_netplay_df.groupby(['match_id', 'set', 'rally'])['getpoint_player'].shift(-1)
    
    # 增加一欄 `game_phase`，早期（rally <= 21）和晚期（rally > 21）
    chou_short_netplay_df['game_phase'] = chou_short_netplay_df['rally'].apply(lambda x: 'Early' if x <= 21 else 'Late')

    # 篩選條件：對手用撲殺得分
    chou_being_killed_df = chou_short_netplay_df[
        (chou_short_netplay_df['opponent_next_type'] == '殺球') & 
        (chou_short_netplay_df['opponent_next_getpoint_player'] != 'CHOU Tien Chen')
    ]
    
    # 計算比例
    total_short_netplay_count = len(chou_short_netplay_df)
    being_killed_count = len(chou_being_killed_df)
    kill_ratio = being_killed_count / total_short_netplay_count if total_short_netplay_count > 0 else 0

    # 印出比例
    print(f"周天成放短球後被對手撲殺的比例: {kill_ratio:.2%}")

    # 在兩個階段內計算被殺分的比例
    phase_group = chou_short_netplay_df.groupby('game_phase')
    for phase, phase_df in phase_group:
        phase_total = len(phase_df)
        phase_kills = len(chou_being_killed_df[chou_being_killed_df['rally'].isin(phase_df['rally'])])
        phase_kill_ratio = phase_kills / phase_total if phase_total > 0 else 0
        print(f"在{phase}階段內，周天成放短球後被對手撲殺的比例: {phase_kill_ratio:.2%}")

    # 視覺化分析
    fig, ax = plt.subplots(figsize=(8, 6))
    kill_ratios = [len(chou_being_killed_df[chou_being_killed_df['rally'].isin(phase_df['rally'])]) / len(phase_df) if len(phase_df) > 0 else 0
                   for phase, phase_df in phase_group]
    
    ax.bar(['Early', 'Late'], kill_ratios, color=['green', 'red'])
    ax.set_ylabel('被撲殺比例')
    ax.set_xlabel('比賽階段')
    ax.set_title('比賽不同階段中的放短球後被撲殺比例')
    plt.tight_layout()

    # 返回圖型物件，不顯示
    fig
else:
    print("數據框為空！無法進行計算。")

第70題
周天成挑高球的落點分布，是否常造成對手容易得分？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 設定字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 假設 df 是已經存在的 DataFrame
if len(df) > 0:
    # 過濾出周天成挑高球的數據和非空的 getpoint_player
    chou_df = df[(df['player'] == 'CHOU Tien Chen') & (df['type'] == '挑球')].copy()
    print(f"周天成挑高球的數據行數: {len(chou_df)}")
    
    # 過濾落點分布 non-null 確保資料準確性
    chou_df = chou_df.dropna(subset=['landing_area']).copy()
    print(f"有效落點數據行數: {len(chou_df)}")
    
    # 分析挑高球後對手是否得分
    # 假設 getpoint_player 為非 'CHOU Tien Chen' 時代表對手得分
    chou_df['opponent_scores'] = chou_df['getpoint_player'].apply(lambda x: x != 'CHOU Tien Chen').astype(int)

    # 計算每個落點區域對手得分的概率
    landing_area_counts = chou_df['landing_area'].value_counts()
    opponent_scoring_prob = chou_df[chou_df['opponent_scores'] == 1]['landing_area'].value_counts() / landing_area_counts
    opponent_scoring_prob = opponent_scoring_prob.fillna(0)
    
    # 畫圖顯示落點分布與得分概率
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 落點分布
    landing_area_counts.sort_index().plot(kind='bar', ax=axes[0], color='skyblue')
    axes[0].set_title('周天成挑高球的落點分布')
    axes[0].set_xlabel('落點區域')
    axes[0].set_ylabel('頻率')
    
    # 得分概率
    opponent_scoring_prob.sort_index().plot(kind='bar', ax=axes[1], color='lightcoral')
    axes[1].set_title('挑高球落點導致對手得分的概率')
    axes[1].set_xlabel('落點區域')
    axes[1].set_ylabel('得分概率')
    
    plt.tight_layout()
else:
    print("DataFrame 為空，無法進行分析。")

第71題
當周天成採取多拍耐心拉吊策略時的平均勝率。

In [ ]:
import pandas as pd

# 假設 df 已經存在，首先檢查 df 的長度
if len(df) == 0:
    raise ValueError("DataFrame is empty. Cannot perform analysis.")

# 定義多拍耐心拉吊策略
def is_patience_rally(group):
    # 篩選出多拍回合，以平球或挑球為主策略
    is_long_rally = group['ball_round'].max() >= 15
    # 放寬條件，檢查平球或挑球的次數是否超過回合總數的三分之一
    main_strategy = group['type'].isin(['平球', '挑球']).sum() > (1/3) * group['ball_round'].max()
    return is_long_rally and main_strategy

# 篩選符合條件的回合
strategy_rallies = df.groupby(['match_id', 'set', 'rally']).filter(is_patience_rally)

# 檢查是否有符合條件的回合
if len(strategy_rallies) == 0:
    raise ValueError("No rallies match the patience rally strategy criteria.")

# 計算周天成在這些回合中的勝率
# 先獲取每個回合的最後一拍
final_shots = strategy_rallies.groupby(['match_id', 'set', 'rally']).last()

# 檢查周天成是否是得分方（勝率計算）
chou_wins = final_shots['getpoint_player'] == 'CHOU Tien Chen'

# 計算勝率
chou_win_rate = chou_wins.sum() / len(final_shots)

print(f"周天成在採取多拍耐心拉吊策略時的平均勝率為: {chou_win_rate:.2%}")

第72題
分析周天成在不同局（第 1 局、第 2 局、第 3 局）殺球比重變化。

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據框不為空
if len(df) > 0:
    # 篩選出周天成的數據
    chou_df = df[df['player'] == 'CHOU Tien Chen']

    # 計算每局殺球次數
    smash_counts = chou_df[chou_df['type'] == '殺球'].groupby(['match_id', 'set']).size()

    # 計算每局總擊球次數
    total_shots = chou_df.groupby(['match_id', 'set']).size()

    # 計算殺球比重 (殺球次數 / 總擊球次數)
    smash_ratio = (smash_counts / total_shots).fillna(0)

    # 繪製條形圖
    fig, ax = plt.subplots(figsize=(10, 6))
    smash_ratio.plot(kind='bar', ax=ax, color='skyblue')

    # 設置圖表標題和標籤
    ax.set_title('周天成在每局中的殺球比重')
    ax.set_xlabel('比賽ID及局數 (match_id, set)')
    ax.set_ylabel('殺球比重')

    # 顯示數值標籤
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.2f}", (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                    textcoords='offset points')

    plt.xticks(rotation=45)
    plt.tight_layout()
else:
    print("數據框為空，無法進行分析。")

第73題
當比賽節奏加快時（每拍時間縮短），周天成的失誤率是否變高？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 設定字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保不處理空資料集
if len(df) > 0:
    # 轉換 time 的格式，並計算每拍的時間
    df['time'] = pd.to_datetime(df['time'])
    df['rally_duration'] = df.groupby(['match_id', 'set', 'rally'])['time'].diff().dt.total_seconds()

    # 將時間分組範圍設置為3個主要區塊，另外可以根據需要進行調整
    def categorize_duration(duration):
        if pd.isna(duration):
            return '未計算'
        elif duration <= 1:
            return '0-1 秒'
        elif 1 < duration <= 2:
            return '1-2 秒'
        else:
            return '超過 2 秒'

    df['duration_category'] = df['rally_duration'].apply(categorize_duration)

    # 計算失誤數 (lose_reason 非空且 player 為周天成)
    df['is_error'] = (df['player'] == 'CHOU Tien Chen') & df['lose_reason'].notna()

    # 計算每個時間區段的失誤率
    error_rates = df.groupby('duration_category')['is_error'].mean()

    # 畫出不同時間段的失誤對比
    fig, ax = plt.subplots(figsize=(8, 6))
    error_rates.plot(kind='bar', ax=ax, color='skyblue')
    ax.set_title('周天成在各時間區段的失誤率')
    ax.set_xlabel('每拍時間區段')
    ax.set_ylabel('失誤率')
    ax.set_xticklabels(error_rates.index)
    plt.tight_layout()

print(error_rates)

第74題
統計周天成所有「一拍得分」的比例與地點分布。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import platform

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 假設 df 已給自於外部引入
if len(df) > 0:
    # 選取一拍結束，且周天成得分的情況
    first_shot_df = df[(df['ball_round'] == 1) & (df['getpoint_player'] == 'CHOU Tien Chen')]

    # 計算一拍得分的比例
    total_rallies = df.groupby(['match_id', 'set', 'rally']).ngroups
    one_shot_wins = first_shot_df.groupby(['match_id', 'set', 'rally']).ngroups
    proportion = one_shot_wins / total_rallies

    print(f"周天成一拍得分的比例: {proportion:.2%}")

    # 得分的位置分布情況
    landing_areas = first_shot_df['landing_area'].value_counts().sort_index()
    
    # 繪製結果
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(landing_areas.index, landing_areas.values, color='teal')
    ax.set_xlabel('落點區域代碼')
    ax.set_ylabel('一拍得分次數')
    ax.set_title('周天成一拍得分的落點分布')
    ax.set_xticks(landing_areas.index)
    ax.set_xticklabels([str(int(x)) for x in landing_areas.index])
    plt.xticks(rotation=45)
    plt.tight_layout()

    print("落點位置和次數：")
    print(landing_areas)
else:
    print("數據集為空，無法進行分析。")

第75題
當周天成打出反手回擊時，對手下一拍攻擊成功率是多少？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確認數據不為空
if len(df) > 0:
    # 篩選周天成使用反手擊球的數據
    chou_backhand_df = df[(df['player'] == 'CHOU Tien Chen') & (df['backhand'] == 1)]
    
    # 獲取周天成打完後對手的下一拍
    chou_backhand_df['next_getpoint_player'] = chou_backhand_df.groupby(['match_id', 'set', 'rally'])['getpoint_player'].shift(-1)
    chou_backhand_df['next_opponent'] = chou_backhand_df.groupby(['match_id', 'set', 'rally'])['opponent'].shift(-1)
    
    # 計算對手成功得分次數和總進攻次數
    opponent_success_count = chou_backhand_df[chou_backhand_df['next_getpoint_player'] == chou_backhand_df['next_opponent']].shape[0]
    total_opponent_attacks = chou_backhand_df['next_opponent'].notna().sum()
    
    # 計算成功率
    if total_opponent_attacks > 0:
        success_rate = opponent_success_count / total_opponent_attacks
    else:
        success_rate = 0.0

    print(f"當周天成打出反手回擊時，對手在下一拍的攻擊成功率為: {success_rate:.2%}")
else:
    print("資料集為空，無法進行分析。")

第76題
分析周天成在「邊線附近擊球」的失誤率是否偏高。

In [ ]:
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據集不為空
if len(df) == 0:
    raise ValueError("數據集為空。")

# 辨識邊線附近的區域代碼
out_of_bounds_ids = [31, 30, 29, 27, 26, 25, 32, 28]  # 出界區域
side_line_area_ids = [1, 4, 5, 8, 13, 16, 17, 20, 21, 24]  # 邊線附近區域假設

# 採用出界和邊線作為邊線附近區域
side_line_areas = out_of_bounds_ids + side_line_area_ids

# 創建一個包含失誤的 DataFrame：周天成的所有擊球
df_chou = df[df['player'] == 'CHOU Tien Chen']

# 計算邊線附近的失誤率
side_line_faults = df_chou[(df_chou['player_location_area'].isin(side_line_areas)) & (~df_chou['lose_reason'].isna())]
side_line_fault_rate = len(side_line_faults) / len(df_chou[df_chou['player_location_area'].isin(side_line_areas)])

# 計算所有區域的失誤率
all_faults = df_chou[~df_chou['lose_reason'].isna()]
overall_fault_rate_chou = len(all_faults) / len(df_chou)

# 計算所有選手的平均失誤率
all_faults_all_players = df[~df['lose_reason'].isna()]
overall_fault_rate_all_players = len(all_faults_all_players) / len(df)

# 打印失誤率以供比較
print(f"周天成在邊線附近的失誤率: {side_line_fault_rate:.2%}")
print(f"周天成的總失誤率: {overall_fault_rate_chou:.2%}")
print(f"所有選手的平均失誤率: {overall_fault_rate_all_players:.2%}")

# 可視化比較失誤率
fig, ax = plt.subplots(figsize=(10, 6))
rates = [side_line_fault_rate, overall_fault_rate_chou, overall_fault_rate_all_players]
labels = ['邊線附近', '周天成總體', '所有選手平均']

ax.bar(labels, rates, color=['royalblue', 'lightseagreen', 'tomato'])
ax.set_ylabel("失誤率")
ax.set_title("失誤率比較")
ax.set_ylim(0, max(rates) * 1.2)
plt.tight_layout()

plt.legend(['邊線附近', '周天成總體', '所有選手平均'])

第77題
計算對手挑球過短時，周天成球種的選擇統計。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import platform

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 檢查資料是否存在
if len(df) > 0:
    # 定義對手挑球過短 (落在前場區域)
    front_court_zones = [17, 18, 19, 20, 21, 22, 23, 24]
    
    # 篩選對手挑球過短，接著周天成回擊的情境
    filtered_df = df[
        (df['player'] == 'CHOU Tien Chen') & 
        (df['opponent'] == 'Kento MOMOTA') & 
        (df['opponent_type'] == 6) &  # 對手挑球 (player_type用上/下文取得更佳)
        (df['hit_area'].shift(1).isin(front_court_zones))
    ]
    
    if len(filtered_df) > 0:
        # 計算不同球種使用頻率
        shot_type_counts = filtered_df['type'].value_counts()

        # 計算得分效果
        win_shot_type_counts = filtered_df[filtered_df['getpoint_player'] == 'CHOU Tien Chen']['type'].value_counts()

        # 視覺化數據
        fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

        # 使用頻率圖
        ax[0].bar(shot_type_counts.index, shot_type_counts.values, color='b')
        ax[0].set_title('周天成在挑球過短時的球種使用頻率')
        ax[0].set_xlabel('球種')
        ax[0].set_ylabel('頻率')

        # 得分效果圖
        ax[1].bar(win_shot_type_counts.index, win_shot_type_counts.values, color='g')
        ax[1].set_title('周天成在挑球過短時的得分球種')
        ax[1].set_xlabel('球種')
        ax[1].set_ylabel('得分次數')

        plt.tight_layout()
    else:
        print("沒有符合條件的數據供分析。")

第78題
當周天成在「局末追分」情況下是否更依賴殺球？（使用率 vs 分數差）

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據量足夠
if len(df) > 0:
    # 確認局末追分 - 修改原數據，為贏家+1分（同時保持 df 原封）
    df['final_score_condition'] = df.groupby(['match_id', 'set'])['player_score'].transform('max')

    # 保留每局的最後一球，並確認贏家
    df_end = df.groupby(['match_id', 'set', 'rally']).apply(lambda x: x[x['ball_round'] == x['ball_round'].max()])
    df_end = df_end.dropna(subset=['getpoint_player'])  # 移除沒有記錄得分者的情況

    # 探索局末追分狀況
    # score_status < -3 代表 CHOU Tien Chen 落後3分以上
    df_trailing = df[(df['score_status'] < -3) & (df['player'] == 'CHOU Tien Chen')]
    
    # 過濾出使用殺球的情況
    df_smash = df_trailing[df_trailing['type'] == '殺球']

    # 計算每個分數差下殺球使用率
    score_diff_usage = df_trailing.groupby('score_status').agg(
        total_counts=('type', 'size'),
        smash_counts=('type', lambda x: (x == '殺球').sum())
    )
    score_diff_usage['smash_usage_rate'] = score_diff_usage['smash_counts'] / score_diff_usage['total_counts']

    # 視覺化
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(score_diff_usage.index, score_diff_usage['smash_usage_rate'], marker='o')
    ax.set_title('周天成在比賽局末追分情況下的殺球使用率與分數差')
    ax.set_xlabel('分數差 (周天成落後)')
    ax.set_ylabel('殺球使用率')
    ax.grid(True)

    plt.tight_layout()
    print(score_diff_usage)
else:
    print("資料不充分")

第79題
統計周天成經常「從防守轉攻擊得分」的常見回合模式。

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 假設 df 已經存在
if len(df) == 0:
    raise ValueError("DataFrame is empty")

# 定義防守和進攻區域
defensive_zones = [1, 2, 3, 4]
offensive_zones_mid = [5, 6, 7, 8, 9, 10, 11, 12]
offensive_zones_front = [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

# 標記哪些是防守球和進攻球
df['is_defensive'] = df['player_location_area'].apply(lambda x: x in defensive_zones)
df['is_offensive'] = df['player_location_area'].apply(lambda x: x in offensive_zones_mid + offensive_zones_front)

# Shift 以識別從防守到進攻的轉換
df['next_is_offensive'] = df.groupby(['match_id', 'set', 'rally'])['is_offensive'].shift(-1)
df['shifted_player'] = df.groupby(['match_id', 'set', 'rally'])['player'].shift(-1)

# 篩選出周天成從防守轉為進攻的記錄
transition_df = df[
    (df['player'] == 'CHOU Tien Chen') & 
    (df['is_defensive']) & 
    (df['next_is_offensive']) & 
    (df['shifted_player'] == 'CHOU Tien Chen')
]

# 篩選出贏得該回合的記錄
winning_df = df[
    (df['player'] == df['getpoint_player']) & 
    (df['getpoint_player'] == 'CHOU Tien Chen')
]

# 進行合併，確保資料不為空
if len(transition_df) > 0 and len(winning_df) > 0:
    result_df = pd.merge(transition_df, winning_df, on=['match_id', 'set', 'rally'])
    
    # 計算方式出現次數
    method_counts = result_df['type_x'].value_counts()
    
    print("周天成從防守轉攻擊並得分的手段:")
    print(method_counts)
    
    # 可視化
    fig, ax = plt.subplots(figsize=(10, 6))
    method_counts.plot(kind='bar', ax=ax)
    ax.set_xlabel('擊球方式')
    ax.set_ylabel('次數')
    ax.set_title('周天成從防守轉攻擊並得分的手段頻率')
    plt.tight_layout()
    # 返回 fig 物件以供進一步操作
    fig
else:
    print("無法找到從防守轉攻擊並得分的記錄。")

第80題
分析周天成在壓迫對手前場後，選擇吊後場 vs 搶網得分的效率比較。

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確認數據集是否存在資料
if len(df) == 0:
    print("數據集為空，無法進行分析。")
else:
    # 處理遺失值
    relevant_columns = ['player', 'getpoint_player', 'type', 'landing_area', 'opponent_location_area']
    df = df.dropna(subset=relevant_columns)

    # 識別「壓迫前場」情況
    df['opponent_previous_location_area'] = df.groupby(['match_id', 'set', 'rally'])['opponent_location_area'].shift(1)
    
    # 定義 zone for 前場 and 後場
    front_court_zones = list(range(17, 25))
    back_court_zones = list(range(1, 5))

    # 找出「吊後場策略」
    df_diaoback = df[
        (df['player'] == 'CHOU Tien Chen') &
        (df['opponent_previous_location_area'].isin(front_court_zones)) & # 壓迫對手前場
        (df['landing_area'].isin(back_court_zones))  # 吊後場
    ]

    # 找出「搶網得分策略」
    df_qiangwang = df[
        (df['player'] == 'CHOU Tien Chen') &
        (df['opponent_previous_location_area'].isin(front_court_zones)) & # 壓迫對手前場
        (df['landing_area'].isin(front_court_zones))  # 搶網
    ]

    # 計算各策略的情況
    diaoback_wins = df_diaoback[df_diaoback['getpoint_player'] == 'CHOU Tien Chen'].shape[0]
    diaoback_total = df_diaoback.shape[0]
    diaoback_errors = df_diaoback[df_diaoback['lose_reason'].notna()].shape[0]

    qiangwang_wins = df_qiangwang[df_qiangwang['getpoint_player'] == 'CHOU Tien Chen'].shape[0]
    qiangwang_total = df_qiangwang.shape[0]
    qiangwang_errors = df_qiangwang[df_qiangwang['lose_reason'].notna()].shape[0]

    # 印出結果
    print(f"吊後場策略: 總數={diaoback_total}, 得分={diaoback_wins}, 失誤={diaoback_errors}")
    print(f"搶網策略: 總數={qiangwang_total}, 得分={qiangwang_wins}, 失誤={qiangwang_errors}")

    # 檢查並計算得分率和失誤率，避免 division by zero
    diaoback_score_rate = diaoback_wins / diaoback_total if diaoback_total > 0 else 0
    diaoback_error_rate = diaoback_errors / diaoback_total if diaoback_total > 0 else 0

    qiangwang_score_rate = qiangwang_wins / qiangwang_total if qiangwang_total > 0 else 0
    qiangwang_error_rate = qiangwang_errors / qiangwang_total if qiangwang_total > 0 else 0

    # 繪圖
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    # 吊後場得分與失誤
    ax[0].bar(['得分', '失誤'], [diaoback_wins, diaoback_errors], color=['green', 'red'])
    ax[0].set_title("吊後場策略")

    # 搶網得分與失誤
    ax[1].bar(['得分', '失誤'], [qiangwang_wins, qiangwang_errors], color=['green', 'red'])
    ax[1].set_title("搶網策略")

    # 繪製配置
    plt.suptitle('策略效能比較')
    plt.tight_layout()
    
    # 顯示圖例
    print(f"吊後場策略詳細數據: 總數={diaoback_total}, 得分率={diaoback_score_rate:.2%}, 失誤率={diaoback_error_rate:.2%}")
    print(f"搶網策略詳細數據: 總數={qiangwang_total}, 得分率={qiangwang_score_rate:.2%}, 失誤率={qiangwang_error_rate:.2%}")

第81題
分析對手如果腳受傷，周天成應該採取哪些進攻策略方式？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 設置字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確認資料存在
if len(df) > 0:
    # 分析獲得分數情況
    # 選擇周天成作為得分者, 且確認是主動得分
    chou_scores = df[(df['getpoint_player'] == 'CHOU Tien Chen') & (df['player'] == 'CHOU Tien Chen')]

    # 假設對手腳受傷，分析周天成應該攻擊的區域
    # 這裡假設腳受傷的對手可能在移動上有困難，因此攻擊場地的邊角可能更有效
    # 我們可以假設 landing_area 1, 5, 9, 13 是場地的邊角
    corner_areas = [1, 5, 9, 13]
    corner_scores = chou_scores[chou_scores['landing_area'].isin(corner_areas)]
    corner_area_counts = corner_scores['landing_area'].value_counts().sort_index()

    # 如果身體方便，可以用柱狀圖視覺化得分最多的區域
    fig, ax = plt.subplots(figsize=(10, 6))
    corner_area_counts.plot(kind='bar', ax=ax)
    ax.set_title('周天成在場地邊角的得分次數')
    ax.set_xlabel('區域')
    ax.set_ylabel('得分次數')
    plt.xticks(rotation=0)
    plt.tight_layout()

    # 打印分析結果
    print("周天成在場地邊角的得分次數:\n", corner_area_counts)

    # 確認周天成是主動得分，並分析其擊球方式
    shot_type_counts = chou_scores['type'].value_counts()

    # 產生擊球方式的直方圖
    fig, ax = plt.subplots(figsize=(10, 6))
    shot_type_counts.plot(kind='bar', ax=ax)
    ax.set_title('周天成不同擊球方式得分次數')
    ax.set_xlabel('擊球方式')
    ax.set_ylabel('得分次數')
    plt.tight_layout()

    # 打印分析結果
    print("周天成不同擊球方式得分次數:\n", shot_type_counts)

    # 分析周天成獲得分的拍數分佈
    rally_counts = chou_scores['rally'].value_counts()

    # 產生節奏變化的圖表
    fig, ax = plt.subplots(figsize=(10, 6))
    rally_counts.plot(kind='line', ax=ax)
    ax.set_title('周天成不同節奏進攻得分次數')
    ax.set_xlabel('拍數')
    ax.set_ylabel('得分次數')
    plt.tight_layout()

    # 打印分析結果
    print("周天成不同節奏得分次數:\n", rally_counts)

第82題
根據對手的殺球，周天成應該如何布置接殺站位與防守策略？

In [ ]:
import platform
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 1. 確認數據量
if len(df) > 0:
    
    # 2. 篩選出 Kento MOMOTA 的殺球數據
    kento_smashed_df = df[(df['player'] == 'Kento MOMOTA') & (df['type'] == '殺球')]

    # 列印確認數據量
    print(f"Kento MOMOTA 殺球次數: {len(kento_smashed_df)}")

    # 3. 確認 landing_x 和 landing_y 沒有遺失值
    kento_smashed_df = kento_smashed_df.dropna(subset=['landing_x', 'landing_y'])

    # 4. 使用熱度圖分析落點區域
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=kento_smashed_df, x='landing_x', y='landing_y', cmap="Reds", fill=True, thresh=0.1)
    plt.title("Kento MOMOTA 殺球落點密集區域")
    plt.xlabel("落點 X 座標")
    plt.ylabel("落點 Y 座標")
    plt.axhline(0, color='black', linewidth=0.8)
    plt.axvline(0, color='black', linewidth=0.8)
    plt.grid(True)
    plt.tight_layout()

    # 顯示熱度圖
    fig = plt.gcf()
    fig.savefig("kento_smash_heatmap.png")  # Save the figure if needed

    # 5. 提出防守策略建議
    # 基於殺球落點位置的分析，建議周天成站在中場偏後區域 (如位置6-10) 處於接球準備的狀態
    # 跨步行動以便快速回防下次攻擊，特別注意網前球與兩側區域的防守

else:
    print("數據集中沒有可用的數據。")

第83題
當在關鍵分，發現對手的體力下降時，周天成應該如何調整自己的進攻節奏與球種選擇？

In [ ]:
import pandas as pd

# 確保字體設置正確，支持中文
import platform
import matplotlib.pyplot as plt
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 假設 df 是我們要分析的數據
if len(df) > 0:
    # 過濾出周天成在「關鍵分」的數據
    key_points_df = df[((df['score_status'] >= -3) & (df['score_status'] <= 3)) & (df['player'] == 'CHOU Tien Chen')]
    
    # 確定周天成得分的球種類型 (主動得分)
    chou_scoring_shots = key_points_df[(key_points_df['getpoint_player'] == 'CHOU Tien Chen') & (key_points_df['player'] == 'CHOU Tien Chen')]

    # 計算不同球種的得分次數
    shot_counts = chou_scoring_shots['type'].value_counts()

    # 打印結果
    print("周天成在關鍵分中的得分球種分佈：")
    print(shot_counts)

    # 繪製球種得分分佈圖
    plt.figure(figsize=(10, 6))
    shot_counts.plot(kind='bar', color='skyblue', alpha=0.8)
    plt.title('周天成在關鍵分中得分的球種分佈')
    plt.xlabel('球種')
    plt.ylabel('得分次數')
    plt.xticks(rotation=45)
    plt.tight_layout()
    fig = plt.gcf()  # 獲取當前圖像對象

else:
    print("數據集中沒有可供分析的數據。")

# 返回推薦策略（假設這是分析後的結果）
print("周天成在關鍵分時應該增加使用的球種：殺球、切球")
print("並保持活躍進攻，利用對手疲勞增加主動得分機會。")

第84題
當對手後場移動速度下降時，周天成應該如何利用球路變化（後場長球、突然直線殺球、前後調動）來持續施壓，避免對手有喘息時間？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 首先檢查數據長度
if len(df) > 0:
    # 計算每回合中對手的移動距離
    df['opponent_move_distance'] = (df['opponent_move_x']**2 + df['opponent_move_y']**2) ** 0.5

    # 選取對手後場移動較慢情況的數據
    # 假設移動距離小於某一閾值表示移動緩慢，這裡先設為 0.1
    slow_opponent_df = df[(df['opponent_move_distance'] < 0.1) & 
                          (df['opponent_location_area'].isin([1, 2, 3, 4]))]

    # 分析周天成的不同球路變化策略
    chou_strategies = {
        '後場長球': slow_opponent_df[(slow_opponent_df['player'] == 'CHOU Tien Chen') & 
                                    (slow_opponent_df['landing_area'].isin([1, 2, 3, 4])) & 
                                    (slow_opponent_df['type'] == '長球')],
        
        '突然直線殺球': slow_opponent_df[(slow_opponent_df['player'] == 'CHOU Tien Chen') & 
                                       (slow_opponent_df['type'] == '殺球') & 
                                       (~slow_opponent_df['landing_area'].isin([1, 2, 3, 4]))],
        
        '前後調動': slow_opponent_df[(slow_opponent_df['player'] == 'CHOU Tien Chen') &
                                    (slow_opponent_df['opponent_location_area'].shift(-1).isin([1, 2, 3, 4])) &
                                    (slow_opponent_df['landing_area'].isin([21, 22, 23, 24]))]  # 落點在前場

    }

    # 分別計算不同策略的成功率
    strategy_results = {}
    for strategy, data in chou_strategies.items():
        # 確認得分情況
        successful_plays = data[data['getpoint_player'] == 'CHOU Tien Chen']
        strategy_results[strategy] = {
            'total_plays': len(data),
            'successful_plays': len(successful_plays),
            'success_rate': len(successful_plays) / len(data) if len(data) > 0 else 0
        }

    # 打印策略結果
    for strategy, result in strategy_results.items():
        print(f"策略: {strategy} -> 總次數: {result['total_plays']}, 成功次數: {result['successful_plays']}, 成功率: {result['success_rate']:.2%}")

# 在視覺化應用例子中，分析不同策略的成功率的對比
fig, ax = plt.subplots(figsize=(10, 6))
strategies = list(strategy_results.keys())
success_rates = [result['success_rate'] for result in strategy_results.values()]
ax.bar(strategies, success_rates, color=['skyblue', 'orange', 'green'])
ax.set_ylabel('成功率 (%)')
ax.set_title('周天成不同球路策略的成功率比較 (對手移動緩慢時)')
plt.tight_layout()

fig

第85題
對手若習慣在第三拍主動攻擊，周天成應該如何設計接發策略，在前三拍就破壞對手節奏並轉守為攻？

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 確保有數據
if len(df) > 0:
    # 根據前三拍篩選數據
    df_first_three_rounds = df[df['ball_round'].isin([1, 2, 3])]

    # 找尋對手在 ball_round == 3 常用攻擊球種和落點
    opponent_third_shot = df_first_three_rounds[
        (df_first_three_rounds['ball_round'] == 3) & 
        (df_first_three_rounds['player'] != 'CHOU Tien Chen')
    ]

    # 計算對手常用球種
    opponent_third_shot_type = opponent_third_shot['type'].value_counts()

    # 計算對手常用落點
    opponent_third_shot_landing_area = opponent_third_shot['landing_area'].value_counts()

    print("對手第三拍常用球種:")
    print(opponent_third_shot_type)

    print("對手第三拍落點分佈:")
    print(opponent_third_shot_landing_area)

    # 設計策略後圖形化表示
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    opponent_third_shot_type.plot(kind='bar', title='對手第三拍常用球種')
    plt.xlabel('球種')
    plt.ylabel('次數')

    plt.subplot(1, 2, 2)
    opponent_third_shot_landing_area.plot(kind='bar', title='對手第三拍落點分佈')
    plt.xlabel('落點')
    plt.ylabel('次數')

    plt.tight_layout()
    plt.show()
else:
    print("Dataset is empty.")

第86題
若對手在回合中主動進攻，周天成可以如何利用「節奏」與「落點深度」讓對手無法進入舒服的進攻狀態？

In [ ]:
import platform
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 字體設置
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據有值
if len(df) > 0:
    # 篩選周天成的回球行為
    chou_tien_chen_actions = df[(df['player'] == 'CHOU Tien Chen')]

    # 過濾掉對手的攻擊球種數據：平球、殺球等以進攻為主的技術
    aggressive_shots = ['平球', '殺球']
    opponent_aggressive_df = chou_tien_chen_actions[chou_tien_chen_actions['opponent_type'].isin([7, 4])]
    
    # 取得周天成回球的落點分佈
    chou_tien_chen_landing_areas = opponent_aggressive_df.groupby('landing_area').size().reset_index(name='counts')

    # 可視化周天成不同落點區域的分佈，匹配這些區域對於干擾對手的有效性
    fig, ax = plt.subplots()
    sns.barplot(data=chou_tien_chen_landing_areas, x='landing_area', y='counts', ax=ax)
    ax.set_title('周天成在對手進攻時的回球落點分佈')
    ax.set_xlabel('落點區域')
    ax.set_ylabel('次數')
    plt.tight_layout()

    # 分析周天成的技術溝通，如何影響對手接下來的回球質量
    # 這裡需要進一步的數據分析來評估節奏和落點深度的影響
    # 例如：分析不同落點深度對對手失誤率的影響
    # 這部分需要具體的數據和分析方法

    # 展示圖表
    plt.show()

else:
    print("數據集為空，無法分析周天成的回球策略。")

第87題
在對手打網前球很精準時，周天成應如何改變網前球的球質與連接方式，減少對手打網前球的頻率？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 設定字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保DataFrame非空
if len(df) > 0:
    # 在完整數據集上創建新欄位以便檢查前一拍的球種
    df['previous_type'] = df.groupby(['match_id', 'set', 'rally'])['type'].shift(1)

    # 過濾出周天成面對對手網前球的情況，即對手打網前球(present)而非周天成
    chou_df = df[(df['opponent'] == 'CHOU Tien Chen') & (df['previous_type'] == '網前球')]
    
    # 打印過濾後的數據量
    print(f"周天成面對網前球的數據量: {len(chou_df)}")

    # 分析不同的球種在對手打網前球後的影響
    impact_df = chou_df.groupby('type')['previous_type'].value_counts(normalize=True).unstack().fillna(0)

    # 打印分析結果
    print("不同球種對網前球的影響:")
    print(impact_df)

    # 周天成在網前球后的連接方式影響
    chou_continuity_df = df[(df['player'] == 'CHOU Tien Chen')]

    # 計算不同球種連接後周天成使用網前球的比例
    continuity_impact_df = chou_continuity_df.groupby('previous_type')['type'].value_counts(normalize=True).unstack().fillna(0)

    # 打印連接方式的影響
    print("連接方式對網前球的影響:")
    print(continuity_impact_df)

    # 視覺化影響
    fig, ax = plt.subplots(figsize=(12, 6))
    impact_df.loc[:, '網前球'].plot(kind='bar', ax=ax)
    ax.set_title('不同球種對對手網前球的影響')
    ax.set_xlabel('對手上一拍使用的球種')
    ax.set_ylabel('對手使用網前球的比例')
    plt.tight_layout()

    # 確保連接方式的圖形
    fig, ax = plt.subplots(figsize=(12, 6))
    continuity_impact_df.loc[:, '網前球'].plot(kind='bar', ax=ax)
    ax.set_title('周天成不同球種連接后使用網前球的比例')
    ax.set_xlabel('周天成上一拍使用的球種')
    ax.set_ylabel('使用網前球的比例')
    plt.tight_layout()

else:
    print("DataFrame is empty.")

第88題
當對手的後場防守能力較弱時，周天成應該如何調整自己的進攻策略與球種選擇，以最大化得分機會？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 設定字體以支持繁體中文顯示
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據量有效
if len(df) > 0:
    # 確認數據不為空，並將空值過濾
    relevant_df = df.dropna(subset=['landing_area', 'lose_reason'])
    
    # 過濾對手在後場的數據(後場區域1-4)的防守情況
    opponent_weakness_df = relevant_df[
        (relevant_df['opponent'] == 'Kento MOMOTA') & 
        (relevant_df['landing_area'].isin([1, 2, 3, 4])) &
        (relevant_df['lose_reason'].notnull())
    ]
    
    # 打印對手後場失分情況
    print("Opponent's weakness (Backcourt Losses):")
    print(opponent_weakness_df[['match_id', 'set', 'rally', 'lose_reason']].head())
    
    # 確認周天成的有效攻擊策略數據
    zhoucheng_attack_df = df[
        (df['player'] == 'CHOU Tien Chen') & 
        (df['getpoint_player'] == 'CHOU Tien Chen') &
        (df['type'].notnull())
    ]

    # 計算周天成使用各種球種的得分次數
    ball_type_success = zhoucheng_attack_df['type'].value_counts()

    # 打印結果
    print("Ball type success count by 周天成:")
    print(ball_type_success)

    # 初始化圖表
    fig, ax = plt.subplots(figsize=(10, 6))

    # 繪製周天成的球種得分情況
    ball_type_success.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')

    # 設置圖表標題和軸標籤
    ax.set_title('周天成各球種得分頻率', fontsize=14)
    ax.set_xlabel('球種', fontsize=12)
    ax.set_ylabel('得分數量', fontsize=12)

    # 確保圖表不重疊
    plt.xticks(rotation=45)
    plt.tight_layout()

    # 顯示圖表
    fig
else:
    print("DataFrame is empty.")

第89題
當周天成發現自己較容易掛網時，應該如何調整自己的擊球策略與站位，以減少掛網失誤的發生？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據集不為空
if len(df) == 0:
    raise ValueError("資料集為空，無法進行分析")

# 篩選周天成的掛網失誤
chou_net_errors = df[(df['player'] == 'CHOU Tien Chen') & (df['lose_reason'] == '掛網')]

# 確認有無掛網數據
if len(chou_net_errors) == 0:
    raise ValueError("資料集中沒有周天成掛網失誤的數據，無法進行分析")

# 分析掛網的球種和擊球高度
net_error_types = chou_net_errors['type'].value_counts()
net_error_hit_height = chou_net_errors['hit_height'].value_counts()

print("掛網球種分析:")
print(net_error_types)

print("\n掛網擊球高度分析:")
print(net_error_hit_height)

# 分析掛網的擊球和落點座標
chou_net_errors_coords = chou_net_errors[['hit_x', 'hit_y', 'landing_x', 'landing_y', 'ball_distance']]

# 輸出部分結果以確認
print(chou_net_errors_coords.head())

# 找出掛網的站位區域
net_error_locations = chou_net_errors['player_location_area'].value_counts()

print("掛網時的站位分布:")
print(net_error_locations)

# 確認df中進行分析的必要欄位是否存在
required_columns = ['player_location_area', 'player_location_x', 'player_location_y', 'moving_x', 'moving_y']
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"缺少必要欄位: {col}")

# 分析站位和落位的搭配問題（無 player_moving_area，需用其他相關欄位）
location_moving_analysis = chou_net_errors[['player_location_area', 'player_location_x', 'player_location_y', 'moving_x', 'moving_y']].groupby(['player_location_area']).count()
print("站位和移動點分析:")
print(location_moving_analysis)

第90題
當周天成在比賽中感覺體力下降時，應該如何調整自己的打法與策略，以保持競爭力並減少失誤？

In [ ]:
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據不為空
if len(df) == 0:
    raise ValueError("The DataFrame is empty.")

# 只篩選周天成的數據
chou_df = df[df['player'] == 'CHOU Tien Chen']

# 計算各個rally中的擊球頻率
rally_counts = chou_df.groupby(['match_id', 'set', 'rally']).size()

# 計算擊球力度(球的飛行距離)
chou_df['calculated_ball_distance'] = np.sqrt((chou_df['landing_x'] - chou_df['hit_x'])**2 + (chou_df['landing_y'] - chou_df['hit_y'])**2)

# 計算跑動距離
chou_df['movement_distance'] = np.sqrt(chou_df['player_move_x']**2 + chou_df['player_move_y']**2)

# 建立 plot 的 x 軸序列
rallies_index = chou_df.groupby(['match_id', 'set', 'rally']).ngroup()

# 可視化：繪製不同數據點的變化
fig, axs = plt.subplots(3, figsize=(12, 18))

# 第一個圖: 击球频率变化
axs[0].plot(rallies_index.unique(), rally_counts, label='擊球頻率')
axs[0].set_title('擊球頻率隨時間的變化', fontsize=14)
axs[0].set_xlabel('回合序號', fontsize=12)
axs[0].set_ylabel('擊球頻率', fontsize=12)
axs[0].legend()

# 第二個圖: 击球力度变化
axs[1].plot(rallies_index, chou_df['calculated_ball_distance'], label='擊球力度', color='green')
axs[1].set_title('擊球力度隨時間的變化', fontsize=14)
axs[1].set_xlabel('回合序號', fontsize=12)
axs[1].set_ylabel('擊球力度 (距離)', fontsize=12)
axs[1].legend()

# 第三个图: 跑动距离变化
axs[2].plot(rallies_index, chou_df['movement_distance'], label='跑動距離', color='red')
axs[2].set_title('跑動距離隨時間的變化', fontsize=14)
axs[2].set_xlabel('回合序號', fontsize=12)
axs[2].set_ylabel('跑動距離', fontsize=12)
axs[2].legend()

plt.tight_layout()

第91題
若對手的右後場出球不穩，周天成應如何調整策略，攻擊其弱側？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 設定字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 假設 df 已經定義且長度大於 0
if len(df) > 0:
    # Step 1: 找到對手位於右後場區域進行擊球的表現
    right_backcourt_zones = [3, 4]
    opponent_right_backcourt = df[df['opponent_location_area'].isin(right_backcourt_zones)]

    # Step 2: 檢查對手在這些區域的失誤
    right_backcourt_errors = opponent_right_backcourt[opponent_right_backcourt['lose_reason'].notna()]

    if not right_backcourt_errors.empty:
        print(f"對手在右後場失誤情形:\n{right_backcourt_errors[['opponent_location_area', 'lose_reason']].value_counts()}")

        # Step 3: 檢查失誤前周天成的擊球策略
        errors_rally_info = right_backcourt_errors[['match_id', 'set', 'rally']].drop_duplicates()

        chou_errors_strategies = df.merge(errors_rally_info, on=['match_id', 'set', 'rally'])
        prev_strategy_df = chou_errors_strategies.groupby(['match_id', 'set', 'rally']).shift(1)
        prev_strategy_df = prev_strategy_df[(prev_strategy_df['player'] == 'CHOU Tien Chen') & (prev_strategy_df['getpoint_player'].notna())]

        if not prev_strategy_df.empty:
            # 檢查周天成採取的策略與其得分的相關性
            chou_scoring_strategies = prev_strategy_df[prev_strategy_df['getpoint_player'] == 'CHOU Tien Chen']
            strategy_counts = chou_scoring_strategies['type'].value_counts()

            print("周天成有效的策略得分情形：")
            print(strategy_counts)

            # 繪製策略與得分情形的圖表
            fig, ax = plt.subplots(figsize=(10, 6))
            strategy_counts.plot(kind='bar', ax=ax)
            ax.set_title('周天成在對手右後場失誤前的有效策略')
            ax.set_xlabel('擊球策略')
            ax.set_ylabel('得分次數')
            plt.xticks(rotation=45)
            plt.tight_layout()
        else:
            print("無法找到周天成在對手失誤前的有效策略訊息。")
    else:
        print("在對手右後場的失誤數據為空，無法進行進一步分析。")
else:
    print("數據框為空，無法進行分析。")

第92題
周天成應如何透過短拍數進攻、節奏加速策略避免進入長回合？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 驗證數據量
if len(df) <= 0:
    raise ValueError("Dataframe is empty.")

# 1. 針對周天成的數據進行篩選
df_chou = df[df['player'] == 'CHOU Tien Chen']

# 2. 定義短拍數和加速節奏策略
# 短拍數: ball_round <= 3
# 加速節奏策略的球種: 殺球 (type == '殺球'), 推撲球 (type == '推撲球')

short_rallies = df_chou[df_chou['ball_round'] <= 3]
aggressive_shots = short_rallies[(short_rallies['type'].isin(['殺球', '推撲球']))]

# 3. 加入一列計算該球是否得分
aggressive_shots['scoring_shot'] = aggressive_shots['player'] == aggressive_shots['getpoint_player']

# 4. 計算策略使用頻率和成功率
total_short_rallies = short_rallies.groupby(['match_id', 'set', 'rally']).ngroups
total_aggressive_rallies = aggressive_shots.groupby(['match_id', 'set', 'rally']).ngroups
successful_aggressive_rallies = aggressive_shots[aggressive_shots['scoring_shot']].groupby(['match_id', 'set', 'rally']).ngroups

# 資料數據打印
print(f"短拍數總數: {total_short_rallies}")
print(f"使用加速節奏策略的短拍數: {total_aggressive_rallies}")
print(f"成功的加速節奏策略數: {successful_aggressive_rallies}")

# 5. 觀察對手的反應
# 通過分析對手如何回擊這些快速進攻球並記下回擊球種
opponent_reaction = aggressive_shots.groupby(['match_id', 'set', 'rally']).last()  # 取每次進攻最後一拍
opponent_reaction_type = opponent_reaction['opponent_type']  # 觀察對手的回擊球種代碼

# Player type mapping
shot_type_mapping = {
    1: '發短球',
    2: '發長球',
    3: '長球',
    4: '殺球',
    5: '切球',
    6: '挑球',
    7: '平球',
    8: '網前球',
    9: '推撲球',
    10: '接殺防守'
}

opponent_reaction_type = opponent_reaction_type.map(shot_type_mapping)

print("對手的回擊球種:", opponent_reaction_type.value_counts())

# 繪製周天成策略使用頻率和成功率
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].bar(['短拍數總數', '加速策略使用', '加速策略成功'], [total_short_rallies, total_aggressive_rallies, successful_aggressive_rallies])
axes[0].set_title('周天成進攻策略使用情況')
axes[0].set_ylabel('次數')

# 繪製對手回擊策略
opponent_reaction_type.value_counts().plot(kind='bar', ax=axes[1])
axes[1].set_title('對手對周天成策略的回擊球種')
axes[1].set_ylabel('次數')

plt.tight_layout()

第93題
當對手在關鍵分(18分以上)皆呈現非進攻狀態，周天成應如何調整球路設計來強化關鍵分得分率？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 設定字體
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s=='Darwin' else ['Microsoft JhengHei', 'SimHei'] if s=='Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 1. 過濾數據
# 關鍵分數情境：opponent_score >= 18 且周天成正在擊球
relevant_df = df[(df['opponent_score'] >= 18) & (df['player'] == 'CHOU Tien Chen')]

# 2. 確認非進攻狀態的球種
non_aggressive_shots = ['網前球', '挑球', '發短球', '切球']

# 3. 計算成功率
result = {}
for shot in non_aggressive_shots:
    # 篩選特定球種且回合得分球員為周天成的記錄
    shot_df = relevant_df[(relevant_df['type'] == shot)]
    
    # 計算回合數和得分數
    total_shot_count = len(shot_df)
    score_count = shot_df[shot_df['getpoint_player'] == 'CHOU Tien Chen'].shape[0]
    
    # 防止除以零情況
    success_rate = score_count / total_shot_count if total_shot_count > 0 else 0
    result[shot] = (total_shot_count, score_count, success_rate)

# 4. 可視化數據
fig, ax = plt.subplots(figsize=(10, 6))

# 提取資料
labels, total_counts, score_counts, success_rates = zip(*[(k, v[0], v[1], v[2]) for k, v in result.items()])

# 繪製結果
ax.bar(labels, success_rates, color='skyblue')

# 添加逐標籤
ax.set_xlabel('球種', fontsize=12)
ax.set_ylabel('成功率', fontsize=12)
ax.set_title('非進攻球種在關鍵分數中的成功率（周天成）', fontsize=14)
ax.set_ylim(0, 1)  # 確保Y軸在[0,1]之間，方便查看比例

# 在柱狀圖上標註數值
for i, rate in enumerate(success_rates):
    ax.text(i, rate + 0.02, f'{rate:.1%}', ha='center', va='bottom')

plt.tight_layout()

# 列印統計信息
print("球種分析及成功率：")
for shot, (total_count, score_count, success_rate) in result.items():
    print(f'{shot}: 使用次數 = {total_count}, 得分次數 = {score_count}, 成功率 = {success_rate:.2%}')

fig  # 返回圖表對象

第94題
當周天成連續失分三次以上時，應如何透過節奏調整與高成功率球路重建比賽掌控感？

In [ ]:
import pandas as pd
import numpy as np

# 假設 df 是已經定義好的 DataFrame
if len(df) > 0:
    # 新建一個表示得分是否屬於對手連續得分的欄位
    df['opponent_consecutive_points'] = (df['getpoint_player'] == 'Kento MOMOTA').astype(int)

    # 按每局計算連續失分
    df['consecutive_loss'] = df.groupby(['match_id', 'set'])['opponent_consecutive_points'].rolling(window=3).sum().reset_index(drop=True)

    # 找出連續失分3次以上的情況
    df['is_consecutive_loss'] = ((df['consecutive_loss'] >= 3) & (df['player'] == 'CHOU Tien Chen')).astype(int)

    # 在連續失分3次以上之後發生變化的所有球種和步驟
    condition = (df['is_consecutive_loss'] == 1)

    # 計算球種成功率
    success_conditions = (df['getpoint_player'] == 'CHOU Tien Chen') & (df['player'] == 'CHOU Tien Chen')
    success_by_type = df[success_conditions].groupby('type').size()
    total_by_type = df[df['player'] == 'CHOU Tien Chen'].groupby('type').size()
    success_rate_by_type = (success_by_type / total_by_type).fillna(0)

    print("成功率表：\n", success_rate_by_type)

    # 分析場上位置與移動
    positions_and_movements = df.loc[condition, ['player_location_area', 'player_move_area', 'hit_area', 'landing_area']]
    
    if positions_and_movements.empty:
        print("周天成的位置與移動分析：無匹配的數據。")
    else:
        print("周天成的位置與移動分析：\n", positions_and_movements)
else:
    print("數據集中沒有可用的數據行。")

第95題
比賽中若觀察到對手因壓力無法耐心打多拍，周天成可如何透過中場拉吊、穩定球路誘發對手躁進失誤？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確認數據是否可用
if len(df) > 0:
    
    # 定義中場區域
    mid_court_zones = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    
    # 篩選周天成的中場擊球
    chou_mid_court = df[(df['player'] == 'CHOU Tien Chen') & (df['player_location_area'].isin(mid_court_zones))]
    
    # 使用 shift(-1) 確認是否導致對方失誤
    chou_mid_court['opponent_lose_reason_next'] = chou_mid_court.groupby(['match_id', 'set', 'rally'])['lose_reason'].shift(-1)
    
    # 篩選有效誘發對手失誤的情況
    effective_inducement = chou_mid_court[chou_mid_court['opponent_lose_reason_next'].notna()]
    
    # 分析主要技術球種
    type_counts = effective_inducement['type'].value_counts()
    
    # 繪圖
    fig, ax = plt.subplots(figsize=(10, 6))
    type_counts.plot(kind='bar', ax=ax, color='skyblue')
    
    ax.set_title('周天成中場誘發對手失誤的球種分佈')
    ax.set_xlabel('球種')
    ax.set_ylabel('次數')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    print("周天成中場誘發對手失誤的主要技術球種分佈：")
    print(type_counts)
    
else:
    print("數據集為空，無法進行分析。")

第96題
如果周天成發現對手能抓到他所有直線殺球，他應該如何調整殺球角度與變化？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確認數據不為空
if len(df) > 0:
    # 過濾出周天成的直線殺球資料
    chou_killshots_df = df[(df['player'] == 'CHOU Tien Chen') & (df['type'] == '殺球')]

    # 當落點的 X座標 比擊球X座標變化不大視為直線
    chou_straight_killshots_df = chou_killshots_df[(abs(chou_killshots_df['landing_x'] - chou_killshots_df['hit_x']) < 0.1)]
    
    # 確認得分情況
    chou_scoring_killshots = chou_straight_killshots_df[chou_straight_killshots_df['player'] == chou_straight_killshots_df['getpoint_player']]
    print(f"周天成直線殺球得分次數: {chou_scoring_killshots.shape[0]}")

    # 查詢被回球的球種
    chou_straight_killshots_df['opponent_shot_type'] = chou_straight_killshots_df.groupby(['match_id', 'set', 'rally'])['type'].shift(-1)
    
    # 紀錄回球的球種統計
    opponent_response_counts = chou_straight_killshots_df['opponent_shot_type'].value_counts()
    print("對手對周天成直線殺球的回應球種統計:")
    print(opponent_response_counts)

    # 建議調整方式：
    # - 使用其他球種與角度，例如斜線殺球或吊球，增加對手的預測難度
    # - 調整擊球落點增強變化性，避免對手預測
    # - 根據對手常用的回應球種，設計相應的戰術，例如如果對手常用殺球回應，可以準備反擊

else:
    print("資料數據集為空，無法進行分析。")

# 可視化對手對殺球的回應球種
fig, ax = plt.subplots(figsize=(10, 6))
opponent_response_counts.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
ax.set_title('對手對周天成直線殺球的回應球種統計')
ax.set_xlabel('球種名稱')
ax.set_ylabel('次數')
plt.xticks(rotation=45)
plt.tight_layout()

第97題
當周天成在長回合中逐漸感覺腳步變慢，他應如何設計球路避免被對手加速壓制？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 字體設定，適用不同系統
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確認df存在且有數據
if len(df) > 0:
    # 先將 'time' 欄位轉換為 datetime 類型
    df['time'] = pd.to_datetime(df['time'])
    
    # 新增一個欄位計算 player 的移動速度
    df['move_distance'] = np.sqrt(df['player_move_x']**2 + df['player_move_y']**2)
    df['time_diff'] = df.groupby(['match_id', 'set', 'rally'])['time'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()
    df['moving_speed'] = df['move_distance'] / df['time_diff']
    
    # 新增前一步的移動速度
    df['previous_moving_speed'] = df.groupby(['match_id', 'set', 'rally'])['moving_speed'].shift(1)
    
    # 定義長回合的條件
    long_rallies = df[df['ball_round'] > 15]
    long_rallies_chou = long_rallies[long_rallies['player'] == 'CHOU Tien Chen']
    
    # 查找腳步變慢的情況
    slow_down_condition = long_rallies_chou['moving_speed'] < 0.6 * long_rallies_chou['previous_moving_speed']

    # 篩選出長回合中腳步變慢的數據
    slow_down_data = long_rallies_chou[slow_down_condition]
    
    # 統計每種球路在慢下來狀況中的使用頻率與得分表現
    strategy_analysis = slow_down_data.groupby('type').agg(
        count=('type', 'size'),
        win_counts=('getpoint_player', lambda x: np.sum(x == 'CHOU Tien Chen'))
    ).reset_index()

    strategy_analysis['win_rate'] = strategy_analysis['win_counts'] / strategy_analysis['count']
    print(strategy_analysis)

    # 繪製不同球路使用頻率及得分率
    fig, ax1 = plt.subplots(figsize=(10, 6))

    ax1.bar(strategy_analysis['type'], strategy_analysis['count'], color='b', label='使用頻率')
    ax2 = ax1.twinx()
    ax2.plot(strategy_analysis['type'], strategy_analysis['win_rate'], color='r', marker='o', label='得分率')

    ax1.set_xlabel('球種')
    ax1.set_ylabel('使用頻率', color='b')
    ax2.set_ylabel('得分率', color='r')
    ax1.set_title('長回合中腳步變慢時的球路策略分析')
    fig.tight_layout()
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
else:
    print("資料不足，無法進行分析。")

第98題
如果周天成在網前打點一直遭對手抓撲殺，他可以怎麼改變接網方式？

In [ ]:
import pandas as pd
import platform
import matplotlib.pyplot as plt
import seaborn as sns

# 字體設置（根據系統設定字體）
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 1. 確認數據量
if len(df) > 0:
    # 2. 過濾出周天成在網前打點並且對手使用撲殺得分的情況
    net_front_areas = list(range(17, 25))  # 網前區域
    df_chou_net_front = df[(df['player'] == 'CHOU Tien Chen') & (df['hit_area'].isin(net_front_areas))]
    
    # 3. 使用 shift 對拍數的處理
    df_chou_net_front['next_opponent_type'] = df_chou_net_front.groupby(['match_id', 'set', 'rally'])['opponent_type'].shift(-1)
    df_chou_net_front['next_getpoint_player'] = df_chou_net_front.groupby(['match_id', 'set', 'rally'])['getpoint_player'].shift(-1)
    
    # 4. 過濾出被撲殺得分的情況
    df_chou_net_front_pushed = df_chou_net_front[(df_chou_net_front['next_opponent_type'] == 9) & (df_chou_net_front['next_getpoint_player'] != 'CHOU Tien Chen')]

    # 5. 繪製圖表來觀察周天成的接網方式和被撲殺的位置
    if len(df_chou_net_front_pushed) > 0:
        # Plot player location when being pushed
        plt.figure(figsize=(10, 6))
        
        sns.scatterplot(data=df_chou_net_front_pushed, x='player_location_x', y='player_location_y', hue='type', palette='viridis')
        plt.title('周天成在網前被撲殺的位置分佈')
        plt.xlabel('X坐標')
        plt.ylabel('Y坐標')
        plt.grid(True)
        plt.tight_layout()

        # 6. 提示策略以改善此情況
        print("建議策略：觀察周天成在不同位置的移動距離和次數，嘗試在防守時候改善步伐移動及手法")
    else:
        print("未找到周天成在網前被撲殺的數據。")
else:
    print("數據集中沒有可用的數據。")

第99題
如果前一場比賽剛打完體力受損，周天成該如何設計戰術減少移動量仍能得分？

In [ ]:
import platform
import pandas as pd
import matplotlib.pyplot as plt

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確定 df 內有數據
if len(df) > 0:
    # 當前分析使用到的變數
    # 只分析周天成的擊球
    chou_df = df[df['player'] == 'CHOU Tien Chen']
    
    # 找出周天成主動得分的情境
    active_win_df = chou_df[chou_df['player'] == chou_df['getpoint_player']]
    
    # 計算周天成使用各種球種得分的分布
    type_score_distribution = active_win_df['type'].value_counts(normalize=True)

    # 繪製圓餅圖
    fig, ax = plt.subplots()
    type_score_distribution.plot(kind='pie', ax=ax, autopct='%.1f%%', 
                                 labels=[f'{k} ({v:.1f}%)' for k, v in type_score_distribution.items()],
                                 title='周天成各球種得分分布')
    plt.tight_layout()
    
    print("周天成主動得分時所使用的球種分布：")
    print(type_score_distribution)
else:
    print("DataFrame is empty. Please check the input data.")

第100題
若對手不斷分析周天成過往戰術，他應如何在比賽中增加「偽裝」與突然性？

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
s = platform.system()
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC'] if s == 'Darwin' else ['Microsoft JhengHei', 'SimHei'] if s == 'Windows' else ['WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False

# 確保數據集不為空
if len(df) > 0:
    # 過濾出周天成的數據
    chou_df = df[df['player'] == 'CHOU Tien Chen']

    # 計算繞頭球與反手球的使用次數
    aroundhead_count = chou_df['aroundhead'].sum()
    backhand_count = chou_df['backhand'].sum()

    # 計算各球種的使用頻率和得分情況
    shot_type_counts = chou_df['type'].value_counts()
    shot_type_scores = chou_df[chou_df['player'] == chou_df['getpoint_player']]['type'].value_counts()

    # 修正偽裝與突然性策略的得分次數計算
    sudden_strategies_score = chou_df[((chou_df['aroundhead'] == 1) | (chou_df['backhand'] == 1)) & (chou_df['player'] == chou_df['getpoint_player'])].shape[0]

    # 繪製各策略使用頻率及得分比較圖
    fig, ax = plt.subplots(2, 1, figsize=(10, 12))

    # 繪製球種使用頻率
    shot_type_counts.plot(kind='bar', ax=ax[0], color='skyblue', alpha=0.7)
    ax[0].set_title('周天成球種使用頻率')
    ax[0].set_xlabel('球種')
    ax[0].set_ylabel('次數')

    # 繪製球種得分次數
    shot_type_scores.plot(kind='bar', ax=ax[1], color='lightgreen', alpha=0.7)
    ax[1].set_title('周天成各球種得分次數')
    ax[1].set_xlabel('球種')
    ax[1].set_ylabel('得分次數')

    plt.tight_layout()

    print(f"繞頭擊球使用達: {aroundhead_count} 次")
    print(f"反手擊球使用達: {backhand_count} 次")
    print(f"使用偽裝與突然性策略得分次數: {sudden_strategies_score}")
else:
    print("數據集為空，無法進行分析。")